In [0]:
%scala

val df = spark.read.json("/FileStore/tables/example.json")
df.printSchema



root
-- array: array (nullable = true)
 |-- element: long (containsNull = true)
-- dict: struct (nullable = true)
 |-- extra_key: string (nullable = true)
 |-- key: string (nullable = true)
-- int: long (nullable = true)
-- string: string (nullable = true)

df: org.apache.spark.sql.DataFrame = [array: array<bigint>, dict: struct<extra_key: string, key: string> ... 2 more fields]

In [0]:
%scala

dbutils.fs.put("/tmp/test.json", """
{"string":"string1","int":1,"array":[1,2,3],"dict": {"key": "value1"}}
{"string":"string2","int":2,"array":[2,4,6],"dict": {"key": "value2"}}
{"string":"string3","int":3,"array":[3,6,9],"dict": {"key": "value3", "extra_key": "extra_value3"}}
""", true)

val testJsonData = spark.read.json("/tmp/test.json")

display(testJsonData)

array,dict,int,string
"List(1, 2, 3)","List(null, value1)",1,string1
"List(2, 4, 6)","List(null, value2)",2,string2
"List(3, 6, 9)","List(extra_value3, value3)",3,string3


In [0]:
%scala

dbutils.fs.put("/tmp/multi-line.json", """[
    {"string":"string1","int":1,"array":[1,2,3],"dict": {"key": "value1"}},
    {"string":"string2","int":2,"array":[2,4,6],"dict": {"key": "value2"}},
    {
        "string": "string3",
        "int": 3,
        "array": [
            3,
            6,
            9
        ],
        "dict": {
            "key": "value3",
            "extra_key": "extra_value3"
        }
    }
]""", true)

val mldf = spark.read.option("multiline", "true").json("/tmp/multi-line.json")
mldf.show(false)


Wrote 385 bytes.
+---------+----------------------+---+-------+
array |dict |int|string |
+---------+----------------------+---+-------+
[1, 2, 3]|[, value1] |1 |string1|
[2, 4, 6]|[, value2] |2 |string2|
[3, 6, 9]|[extra_value3, value3]|3 |string3|
+---------+----------------------+---+-------+

mldf: org.apache.spark.sql.DataFrame = [array: array<bigint>, dict: struct<extra_key: string, key: string> ... 2 more fields]

In [0]:
%sql 
CREATE TEMPORARY VIEW multiLineJsonTable
USING json
OPTIONS (path="/tmp/multi-line.json",multiline=true);

select * from multiLineJsonTable;


com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.analysis.TempTableAlreadyExistsException: Temporary view 'multiLineJsonTable' already exists;
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.createTempView(SessionCatalog.scala:602)
	at org.apache.spark.sql.execution.datasources.CreateTempViewUsing.run(ddl.scala:111)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:79)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:234)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3728)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:116)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:248)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:101)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:198)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3726)
	at org.apache.spark.sql.Dataset. (Dataset.scala:234)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:676)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:671)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:144)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:488)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:50)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:277)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:270)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:50)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:465)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
	at com.databricks.backend.daemon.driver.